In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow


KeyboardInterrupt: 

In [ ]:
#Load training data
df = pd.read_csv("cosas_finales\casos_all - obtiene7-10.txt", error_bad_lines=False)
x_train = df.iloc[:, 0:6]

y_train = []
for i in range(0,len(df)):
  high = df.tipo.values[i]
  if high == 0:
    y_train.append([1,0,0,0,0,0])
  elif high == 1:
    y_train.append([0,1,0,0,0,0])
  elif high == 2:
    y_train.append([0,0,1,0,0,0])
  elif high == 3:
    y_train.append([0,0,0,1,0,0])
  elif high == 4:
    y_train.append([0,0,0,0,1,0])
  elif high == 5:
    y_train.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(df.tipo.values[i]))

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.asarray(x_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

x_train

C:\Users\migue\AppData\Local\Temp\ipykernel_15060\2204598858.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("cosas_finales\casos_all - obtiene7-10.txt", error_bad_lines=False)


In [ ]:
#Load validation data
real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)
real_df

x_train_val = real_df.iloc[:, 1:7]
x_train_val

y_train_val = []
for i in range(0,len(real_df)):
  high = real_df.tipo.values[i]
  if high == 0:
    y_train_val.append([1,0,0,0,0,0])
  elif high == 1:
    y_train_val.append([0,1,0,0,0,0])
  elif high == 2:
    y_train_val.append([0,0,1,0,0,0])
  elif high == 3:
    y_train_val.append([0,0,0,1,0,0])
  elif high == 4:
    y_train_val.append([0,0,0,0,1,0])
  elif high == 5:
    y_train_val.append([0,0,0,0,0,1])
  else:
    print(str(i) + " " + str(real_df.tipo.values[i]))

x_train_val, y_train_val = np.array(x_train_val), np.array(y_train_val)
x_train_val = np.asarray(x_train_val).astype('float32')
y_train_val = np.asarray(y_train_val).astype('float32')

x_train_val

C:\Users\migue\AppData\Local\Temp\ipykernel_15060\1008687113.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  real_df = pd.read_csv("cosas_finales/tiempos.csv", error_bad_lines=False)


In [ ]:
model = load_model("cosas_finales\model3-8.h5")
model.summary()

In [ ]:
#Train the model with the 'train' portion of the data 
#The other parameters are tuned down in order to reduce train time but should be higher
callback = EarlyStopping(monitor='val_accuracy', patience = 6, restore_best_weights=True)

model.fit(x_train, y_train, batch_size=8, epochs=20, callbacks=[callback], validation_data=(x_train_val, y_train_val))

Epoch 1/20
14987/97000 [===>..........................] - ETA: 9:18 - loss: 0.6772 - accuracy: 0.6636

In [ ]:
real_df_preds = model.predict(x_train_val)

real_preds = []
for i in range(0,len(real_df_preds)):
  p = real_df_preds[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])

err = real_preds - y_train_val
err

añadir O(1).
t2 - t1 y t3 - t1 => plot (esto no hace falta era para el debug)

In [ ]:
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

error_count = 0
for i in range(0, len(err)):
  if max(err[i]) != min(err[i]):
    print(str(real_df.name.values[i]) + " sol: " + numToType(real_df.tipo.values[i]))
    error_count += 1
print(str(error_count) + " / " + str(len(real_preds)) + " => real error% of " + str(error_count/len(real_preds)*100))

niniosPrimero sol: N
--cazaTesoros sol: N^2
tiroPalindromo sol: N^2
3 / 8 => real error% of 37.5
